In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True


Using matplotlib backend: Qt5Agg


In [2]:
KeyInclude = ['IncreaseTimeDwell', 'DecreaseTimeDwell']
TimeDwellOrig = 800
TimeFixation = 300

In [3]:
def ComputeDwellTime(userKeys):
    #print(userKeys)
    TimeDwellChanges = [key for key in userKeys if key[1] in KeyInclude]
    #print(TimeDwellChanges)
    
    if TimeDwellChanges:
        for pctChange in TimeDwellChanges:
            if pctChange[2] == 1:
                if pctChange[1] == 'IncreaseDwellTime':
                    TimeDwellNew = TimeDwellNew + 100
                else:
                    TimeDwellNew = TimeDwellNew - 100
    else:
        TimeDwellNew = TimeDwellOrig
    #print(TimeDwellNew)   
    return TimeDwellNew

In [4]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtScratchPad(keysLookedAt):
    epoch = list()   
    epochList = list()
    
    scratchPadKeyTime = [key for key in keysLookedAt if 'ScratchPad' in key[1]]
    scratchPadInd = [keyInd[0] for keyInd in enumerate(keysLookedAt) if 'ScratchPad' in keysLookedAt[keyInd[0]][1]]
    scratchPadIndArray = np.diff(np.asarray(scratchPadInd))
    
    # TODO: reduce the list to only the first and the last elements of the epoch
    
    # Find unique looks at scratchpad, by checking if progress pct is greater than previous one
    
    for keyInd in enumerate(scratchPadKeyTime):
        if keyInd[0] == 0:
            # first element
            epoch.append(scratchPadKeyTime[keyInd[0]])
            continue
        
        progressPctPrevious = scratchPadKeyTime[keyInd[0]-1][2]
        progressPctNow = scratchPadKeyTime[keyInd[0]][2]
        
        #print(progressPctPrevious, progressPctNow)
        if progressPctPrevious > progressPctNow:
            # new fixation has started
            epochList.append(epoch[0])
            epoch = list()
        
        epoch.append(scratchPadKeyTime[keyInd[0]])
            
    #print(epochList)

    return epochList

In [5]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [6]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [7]:
# function to remove all invalid data in gazelog
def cleanGazeLog(gazeLog):
    gazeLogNew = list()
    for rowInd in enumerate(gazeLog):
        if 'Invalid' not in gazeLog[rowInd[0]]:
            gazeLogNew.append(gazeLog[rowInd[0]])
    return gazeLogNew

In [48]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)  
#     vals2 = vals.replace([np.inf, -np.inf], np.nan).interpolate('linear', limit_area = 'inside')
#     vals3 = vals.replace([np.inf, -np.inf], np.nan).interpolate('quadratic', limit_area = 'inside')
#     vals4 = vals.replace([np.inf, -np.inf], np.nan).interpolate('cubic', limit_area = 'inside')
    
#     fig = plt.figure()
#     ax = fig.add_subplot(1,1,1)
#     ax.plot(vals, '--')
#     ax.plot(vals2, ':')
#     ax.plot(vals3, '-.')
#     #ax.plot(vals4, ':')
    
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    vals = vals0.fillna(vals0.interpolate('linear', limit_area = 'inside')) # linear interpolation instead 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data
    #vals = vals.replace([np.inf, -np.inf], np.nan).fillna(vals.rolling(window=k, min_periods=1, center=True).median())
    #vals = vals.replace([np.inf, -np.inf], np.nan).fillna(method='ffill')
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #valsWoNan = vals.replace([np.inf, -np.inf], np.nan).dropna()
    return(vals)

In [49]:
def plot5sForUniqueScratchPadFixation(scratchPadLookedAtEpoch, scratchPadList, phraseList, GazeLog, TimeDwell, subjName):
    
    flagFirst = 0
    rowTimeList = -1
    phraseScratchedList = list()
    phraseToBeScratchedList = list()
    timeUserLooked = list()
    timeScratchAll = list()
    timeToBeScratchedAll = list()
   
    phraseScratchedInd = 0
    phraseToBeScratchedInd = 0
    Veracity = list()
    
    timeStrUserLooked =  [item[0] for item in scratchPadLookedAtEpoch]
    timeStrScratch = [item1[0] for item1 in scratchPadList]
    timeStrToBeScracthed = [item2[0] for item2 in phraseList]
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    
    timeUserLooked = timeConversion(timeStrUserLooked)
    timeScratchAll = timeConversion(timeStrScratch)
    timeToBeScratchedAll = timeConversion(timeStrToBeScracthed)
    timeGazeLog = timeConversion(timeStrGazeLog)
        
    
    # Create list of pupil sizes from gazelog
    pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in GazeLog]
    pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in GazeLog]

    # Pupil distance (x,y,z) in User Coordinate system 
    pupilDistLx = [float(item6[3]) if 'Invalid' not in item6 else np.nan for item6 in GazeLog]
    pupilDistLy = [float(item7[4]) if 'Invalid' not in item7 else np.nan for item7 in GazeLog]
    pupilDistLz = [float(item8[5]) if 'Invalid' not in item8 else np.nan for item8 in GazeLog]
    pupilDistRx = [float(item9[10]) if 'Invalid' not in item9 else np.nan for item9 in GazeLog]
    pupilDistRy = [float(item10[11]) if 'Invalid' not in item10 else np.nan for item10 in GazeLog]
    pupilDistRz = [float(item11[12]) if 'Invalid' not in item11 else np.nan for item11 in GazeLog]
    
    # create dataframe from the pupil distances
    df_pupilDist = pd.DataFrame(np.column_stack([pupilDistLx, pupilDistLy, pupilDistLz, pupilDistRx, pupilDistRy, pupilDistRz]), columns=['pupilDistLx', 'pupilDistLy', 'pupilDistLz', 'pupilDistRx', 'pupilDistRy', 'pupilDistRz'])
    df_pupilDist = df_pupilDist.interpolate('linear', limit_area = 'inside')
    #print(df_pupilDist.pupilDistLx)
    
    pupilDist = [math.sqrt((df_pupilDist.pupilDistLx[indPt]-df_pupilDist.pupilDistRx[indPt])**2 + (df_pupilDist.pupilDistLy[indPt]-df_pupilDist.pupilDistRy[indPt])**2 + (df_pupilDist.pupilDistLz[indPt]-df_pupilDist.pupilDistRz[indPt])**2) for indPt in range(0, len(pupilDistLx))]
    
    fig = plt.figure()
    axL = fig.add_subplot(3, 1, 1)
    axR = fig.add_subplot(3, 1, 2)
    axDist = fig.add_subplot(3, 1, 3)
    
    #print(timeUserLooked)
    
    for timeList in timeUserLooked:
        rowTimeList = rowTimeList + 1
        # Extract epoch data from gazelog
        
        timeScratchPadActive = timeList - datetime.timedelta(milliseconds=float(scratchPadLookedAtEpoch[rowTimeList][2])*TimeDwell) - datetime.timedelta(milliseconds=TimeFixation)
        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timeScratchPadActive)
        
        timeWindowEnd = timeList + datetime.timedelta(seconds=5)
        
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeWindowEnd)
        #print(GazeLogStartInd, GazeLogEndInd)
        
        if GazeLogStartInd != GazeLogEndInd:
            gazeEpochL = pupilLogL[GazeLogStartInd:GazeLogEndInd]
            gazeEpochR = pupilLogR[GazeLogStartInd:GazeLogEndInd]
            interPupilDist = pupilDist[GazeLogStartInd:GazeLogEndInd]
            
            winSize = 25
            # Filter pupil sizes
            pupilWoOutlierL = hampel(gazeEpochL, winSize, 3)
            pupilWoOutlierR = hampel(gazeEpochR, winSize, 3)
            
            #print(pupilWoOutlierR)
            
            # Moving Mean of data without outliers: 
            pupilMeanL = pupilWoOutlierL.rolling(window=winSize, min_periods=1, center=True).mean()
            pupilMeanR = pupilWoOutlierR.rolling(window=winSize, min_periods=1, center=True).mean()
            
            # After the filtering is done: CHANGE THIS
            pupilSizeL = pupilWoOutlierL.mean(numeric_only=float)[0]
            pupilSizeR = pupilWoOutlierR.mean(numeric_only=float)[0] 
            
            pupilPlotL = (pupilMeanL - pupilMeanL[0][0])/pupilMeanL[0][0]
            pupilPlotR = (pupilMeanR - pupilMeanR[0][0])/pupilMeanR[0][0]
            
            #pupilL = pupilL + pupilPlotL
            #pupilR = pupilR + pupilPlotR
            
        else:
            print('one')
            continue
            #continue # remove samples with only 1 sample
        
        #print(pupilPlotL)
        # Also to check if phrase typed is correct or not, only check the last element of epoch
        #print(timeList[-1])

        
        timeOfPhraseScratchedNow, phraseScratchedInd = nearestTimePoint(timeScratchAll, timeList)
        timeOfPhraseToBeScratchedNow, phraseToBeScratchedInd = nearestTimePoint(timeToBeScratchedAll, timeList)
        
        if phraseScratchedInd < 0:
            phraseScratched = ''
        else:
            phraseScratched = scratchPadList[phraseScratchedInd][1]
        
        phraseToBeScratched = phraseList[phraseToBeScratchedInd][1]
        
        #print(phraseScratched, phraseToBeScratched)
        
        # are they the same? yes/no -> save in list with the time
        if phraseScratched in phraseToBeScratched:
            if 'THE EXPERIMENT IS NOW DONE' in phraseToBeScratched:
                #print('exp done')
                continue
            Veracity.append([pupilSizeL, pupilSizeR, 'C'])
            x = np.arange(0, float(len(pupilPlotL)/90), float(1/90))
            if flagFirst == 0:
                axL.plot(x, pupilPlotL, label = str(rowTimeList)+' C')
                axR.plot(x, pupilPlotR, label = str(rowTimeList)+' C')
                flagFirst = 1
            else:
                axL.plot(x, pupilPlotL, '--', label = str(rowTimeList)+' C')
                axR.plot(x, pupilPlotR, '--', label = str(rowTimeList)+' C')
                
            # Plot inter pupil distance
            axDist.plot(x, interPupilDist, '--', label = str(rowTimeList)+ ' C')
        else:
            Veracity.append([pupilSizeL, pupilSizeR, 'INC'])
            x = np.arange(0, float(len(pupilPlotL)/90), float(1/90))
            if flagFirst == 0:
                axL.plot(x, pupilPlotL, label = str(rowTimeList)+' INC')
                axR.plot(x, pupilPlotR, label = str(rowTimeList)+' INC')
                flagFirst = 1
            else:
                axL.plot(x, pupilPlotL, ':', label = str(rowTimeList)+' INC')
                axR.plot(x, pupilPlotR, ':', label = str(rowTimeList)+' INC')
                
            # Plot inter pupil distance
            axDist.plot(x, interPupilDist, ':', label = str(rowTimeList)+ ' INC')
            
    
    axL.set_title('%s : Left' %subjName)
    axL.set_xlabel('Time [in s]')
    axL.set_ylabel('Relative Pupil dilation [a.u.]')
    axL.legend()
    axR.set_title('%s : Right' %subjName)
    axR.set_xlabel('Time [in s]')
    axR.set_ylabel('Relative Pupil dilation [a.u.]')
    plt.tight_layout()      
    axR.legend()
    return Veracity

In [47]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs and ('jl' in root or 'pt_DT' in root or 'eo' in root or 'mm' in root):
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root or 'ae' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:
                # Compute dwell time
                TimeDwell = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                scratchPadKeyTime = UserLookedAtScratchPad(userKeys)

                # clean gaze log and remove rows with any invalid data -- not clean here but later, and replace nan values with 
                # median values in epoch
                #gazeLogValid = cleanGazeLog(gazeLog)
                # gazeLogValid = gazeLog
                
                # for every element, find the time closest and previous to it, and check what was typed 
                # AND what should have been typed
                # Also, add gaze data to epoch, but first subtract the fixation time and complete the pupil data filtering as
                # per Per's paper
                
                a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                WrittenTruthListAll = plot5sForUniqueScratchPadFixation(scratchPadKeyTime, scratchPad, stimPhrase, gazeLog, TimeDwell, subjName)
                #print('done')
                break


\May15\jl_DT\
            0
0    2.796341
1    2.809875
2    2.828049
3    2.831223
4    2.815323
5    2.772720
6    2.787735
7    2.818024
8    2.868179
9    2.825500
10   2.829819
11   2.802765
12   2.822250
13   2.776886
14   2.776886
15   2.781204
16   2.810135
17   2.802780
18   2.780380
19   2.805466
20   2.813232
21   2.832230
22   2.816422
23   2.816422
24   2.802368
25   2.787109
26   2.818100
27   2.852676
28   2.853622
29   2.834808
..        ...
441  2.852783
442  2.854385
443  2.875595
444  2.875656
445  2.905441
446  2.882874
447  2.880997
448  2.847458
449  2.847214
450  2.812134
451  2.796814
452  2.844177
453  2.844177
454  2.934937
455  3.020935
456  2.906921
457  2.883850
458  2.852844
459  2.847183
460  2.843506
461  2.840729
462  2.836761
463  2.848389
464  2.868530
465  2.875946
466  2.853897
467  2.828918
468  2.756271
469  2.790512
470  2.790512

[471 rows x 1 columns]
VALS             0
0    2.796341
1    2.809875
2    2.828049
3    2.831223
4    2.815323
5    

            0
0    5.010498
1    5.010498
2    5.016495
3    5.036179
4    4.975876
5    4.966553
6    4.956528
7    4.992828
8    4.991699
9    4.993088
10   4.993088
11   4.993057
12   5.002197
13   5.002197
14   4.984756
15   4.956955
16   4.956955
17   4.946823
18   4.943436
19   4.943436
20   4.936050
21   4.919846
22   4.849823
23   4.844818
24   4.824951
25   4.866974
26   4.848083
27   4.830292
28   4.781525
29   4.783630
..        ...
448  5.099380
449  5.097488
450  5.081619
451  5.076477
452  5.066315
453  5.064667
454  5.055206
455  5.060898
456  5.056046
457  5.051514
458  5.044876
459  5.053467
460  5.051056
461  5.051239
462  5.053711
463  5.055542
464  5.052185
465  5.054001
466  5.050568
467  5.054489
468  5.065460
469  5.067062
470  5.067245
471  5.036652
472  5.037140
473  5.035675
474  5.055267
475  5.054474
476  5.062439
477  5.031128

[478 rows x 1 columns]
VALS             0
0    5.010498
1    5.010498
2    5.016495
3    5.036179
4    4.975876
5    4.966553
6    

            0
0         NaN
1         NaN
2    4.869949
3    4.900818
4    4.937408
5    4.979965
6    4.978577
7    4.958359
8    4.970978
9    4.968475
10   4.974762
11   4.981186
12   4.985611
13   4.987061
14   4.998718
15   5.004242
16   5.001129
17   5.002594
18   5.007141
19   5.017349
20   5.012100
21   5.009186
22   5.000549
23   4.996582
24   4.993301
25   4.998032
26   4.996002
27   4.999298
28   4.994568
29   4.998093
..        ...
447       NaN
448       NaN
449       NaN
450       NaN
451       NaN
452       NaN
453       NaN
454  4.747086
455  4.783783
456  4.857132
457  4.966476
458  5.098526
459  5.087708
460  5.070343
461  5.071472
462  5.065033
463  5.061005
464  5.061890
465  5.059631
466  5.053421
467  5.050735
468  5.056076
469  5.063629
470  5.068573
471  5.076996
472  5.095200
473  5.118622
474  5.117722
475  5.109238
476  5.125519

[477 rows x 1 columns]
VALS             0
0         NaN
1         NaN
2    4.869949
3    4.900818
4    4.937408
5    4.979965
6    

VALS             0
0    4.646866
1    4.654114
2    4.654114
3    4.673981
4    4.679626
5    4.679626
6    4.691177
7    4.620697
8    4.620697
9    4.646957
10   4.722626
11   4.722626
12   4.745331
13   4.744339
14   4.744339
15   4.755280
16   4.722809
17   4.722809
18   4.727997
19   4.736603
20   4.736603
21   4.753799
22   4.769279
23   4.784760
24   4.793961
25   4.787643
26   4.787643
27   4.784317
28   4.809448
29   4.809448
..        ...
448  4.793427
449  4.793427
450  4.778625
451  4.780960
452  4.780960
453  4.748306
454  4.745316
455  4.745316
456  4.710983
457  4.707520
458  4.707520
459  4.670563
460  4.669968
461  4.669968
462  4.640228
463  4.616760
464  4.616760
465  4.574860
466  4.558899
467  4.558899
468  4.524582
469  4.583710
470  4.583710
471  4.552979
472  4.520660
473  4.520660
474  4.476852
475  4.458740
476  4.458740
477  4.444000

[478 rows x 1 columns]
            0
0    5.357773
1    5.357773
2    5.343796
3    5.360077
4    5.364258
5    5.370544
6    

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


            0
0    3.874405
1    3.883926
2    3.875870
3    3.890121
4    3.893814
5    3.906555
6    3.904587
7    3.913483
8    3.911926
9    3.933701
10   3.932327
11   3.931931
12   3.927414
13   3.931641
14   3.936768
15   3.944199
16   3.936005
17   3.934616
18   3.929764
19   3.927444
20   3.923813
21   3.924362
22   3.926758
23   3.927109
24   3.924591
25   3.924835
26   3.923691
27   3.923859
28   3.927322
29   3.924103
..        ...
448  3.915863
449  3.925385
450  3.928299
451  3.927490
452  3.922256
453  3.926437
454  3.924530
455  3.941223
456  3.944290
457  3.966705
458  3.991608
459  3.987808
460  3.963013
461  3.938431
462  3.945953
463  3.949707
464  3.942902
465  3.951233
466  3.958466
467  3.963974
468  3.968933
469  3.975281
470  3.978394
471  3.981705
472  3.984756
473  3.981979
474  3.982941
475  3.978821
476  3.984085
477  3.982330

[478 rows x 1 columns]
VALS             0
0    3.874405
1    3.883926
2    3.875870
3    3.890121
4    3.893814
5    3.906555
6    

            0
0    3.983292
1    3.985565
2    3.985367
3    3.981934
4    3.969223
5    3.972733
6    3.973877
7    3.969528
8    3.962814
9    3.958878
10   3.952316
11   3.945282
12   3.945969
13   3.945892
14   3.941879
15   3.950745
16   3.953964
17   3.961487
18   3.963165
19   3.963303
20   3.969986
21   3.961243
22   3.969437
23   3.962662
24   3.971817
25   3.981232
26   3.991364
27   3.998856
28   3.997757
29   4.005310
..        ...
445  4.160019
446  4.169952
447  4.184998
448  4.180847
449  4.187698
450  4.181427
451  4.184799
452  4.192398
453  4.199600
454  4.214737
455  4.219101
456  4.215744
457  4.212143
458  4.238098
459  4.252411
460  4.243637
461  4.222946
462  4.217819
463  4.207001
464  4.204010
465  4.200897
466  4.195404
467  4.198181
468  4.197311
469  4.210938
470  4.209946
471  4.206696
472  4.196823
473  4.198837
474  4.208511

[475 rows x 1 columns]
VALS             0
0    3.983292
1    3.985565
2    3.985367
3    3.981934
4    3.969223
5    3.972733
6    

            0
0         NaN
1         NaN
2         NaN
3         NaN
4    2.386826
5    2.104263
6    3.517395
7    3.555466
8    3.570145
9    3.596176
10   3.614563
11   3.603897
12   3.593307
13   3.579971
14   3.580765
15   3.602081
16   3.609390
17   3.614883
18   3.623108
19   3.634048
20   3.635056
21   3.631363
22   3.643295
23   3.655457
24   3.659607
25   3.671661
26   3.677353
27   3.677368
28   3.697433
29   3.696381
..        ...
448       NaN
449       NaN
450       NaN
451       NaN
452       NaN
453       NaN
454       NaN
455       NaN
456       NaN
457       NaN
458       NaN
459       NaN
460       NaN
461       NaN
462       NaN
463       NaN
464       NaN
465       NaN
466       NaN
467       NaN
468       NaN
469       NaN
470       NaN
471       NaN
472       NaN
473       NaN
474       NaN
475       NaN
476       NaN
477       NaN

[478 rows x 1 columns]
VALS             0
0         NaN
1         NaN
2         NaN
3         NaN
4    2.386826
5    2.104263
6    

            0
0    4.184509
1    4.199783
2    4.205811
3    4.214493
4    4.213089
5    4.215530
6    4.208069
7    4.208206
8    4.205109
9    4.206451
10   4.217392
11   4.211777
12   4.209671
13   4.189606
14   4.180450
15   4.171967
16   4.177139
17   4.165863
18   4.165161
19   4.154999
20   4.150330
21   4.139603
22   4.117371
23   4.095413
24   4.089432
25   4.093170
26   4.088394
27   4.087341
28   4.077896
29   4.065109
..        ...
448  3.875992
449  3.873260
450  3.862228
451  3.872360
452  3.880554
453  3.885406
454  3.881348
455  3.873810
456  3.869064
457  3.857483
458  3.868896
459  3.878143
460  3.884354
461  3.886368
462  3.889404
463  3.894318
464  3.904297
465  3.910904
466  3.923401
467  3.930252
468  3.938309
469  3.942291
470  3.951981
471  3.953537
472  3.950607
473  3.965927
474  3.966553
475  3.970810
476  3.967102
477  3.963882

[478 rows x 1 columns]
VALS             0
0    4.184509
1    4.199783
2    4.205811
3    4.214493
4    4.213089
5    4.215530
6    

            0
0    3.655151
1    3.647934
2    3.637329
3    3.627151
4    3.630936
5    3.633179
6    3.643631
7    3.641418
8    3.635986
9    3.623779
10   3.619583
11   3.618469
12   3.621307
13   3.629578
14   3.637161
15   3.645935
16   3.653519
17   3.663391
18   3.665222
19   3.656433
20   3.657730
21   3.666641
22   3.673721
23   3.682724
24   3.689545
25   3.702194
26   3.703918
27   3.715897
28   3.703964
29   3.713379
..        ...
447  3.952469
448  3.949463
449  3.951935
450  3.956039
451  3.947784
452  3.952011
453  3.950668
454  3.969604
455  3.976105
456  3.983734
457  3.983047
458  3.994247
459  4.006134
460  4.015472
461  4.017578
462  4.023666
463  4.019379
464  4.020187
465  4.032852
466  4.037674
467  4.043045
468  4.053635
469  4.064545
470  4.072372
471  4.070328
472  4.066727
473  4.077255
474  4.096375
475  4.102341
476  4.110641

[477 rows x 1 columns]
VALS             0
0    3.655151
1    3.647934
2    3.637329
3    3.627151
4    3.630936
5    3.633179
6    